## Basis Swap Par Spreads

The goal of the project is to compute the par spreads of a pool of basis swaps, i.e. those contracts in which two floating IBOR rates, with different tenors, are exchanged. The par spread is that margin which has to be paid/received on top of the floating rate with the shorter tenor.

The input parameters are:

* the par rate of a set of Overnight Index Swaps (i.e. their market quotation);
* 3 sets of forward libor rates and their fixing dates, one each for EURIBOR 1M, 3M and 6M;
* the static data of the pool of swaps (nominal, maturity, tenor of the first and second leg).

### Notes and hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for multi-curve evaluation.

Remember to reuse the code we developed during the lessons in `finmarkets.py` as much as possible, instead of trying to rewrite everything from scratch!

When not possible develop your own classes.

## Inputs

[Overnight swap quotes](https://github.com/matteosan1/finance_course/raw/develop/Projects/ois_quotes.xlsx)

[EURIBOR XM (choose right sheet)](https://github.com/matteosan1/finance_course/raw/develop/Projects/euribor_curve.xlsx)

[Basis Swap definition](https://raw.githubusercontent.com/matteosan1/finance_course/develop/Projects/basis_swaps_proj1.csv)

In [ ]:
from datetime import date

pricing_date = date.today()